In [1]:
import os
import glob
import time
import plotly
import requests
import pandas as pd

plotly.offline.init_notebook_mode(connected=True)

## Setup Exam Dates

In [2]:
exam_dates = {
  'EARTH 270': '2017-04-09', # -1, morning exam
  'ECE 318': '2017-04-16',   # -1, morning exam
  'ECE 356': '2017-04-13',
  'ECE 358': '2017-04-10',   # -1, morning exam
  'ECE 390': '2017-04-19'
}

## Trello Integration

In [3]:
# define trello auth parameters
board = ''
applicationKey = ''
token = ''

In [4]:
# Get a list of active courses
endpoint = 'https://api.trello.com/1/board/{}/lists?cards=open&card_fields=all&key={}&token={}'.format(board, applicationKey, token)
r = requests.get(endpoint)

In [5]:
list_id_course_map = {}
card_id_task_map = {}
card_id_list_id_map = {}

for course in r.json():
    list_id_course_map[course['id']] = course['name']
    for card in course['cards']:
        card_id_task_map[card['id']] = card['name']
        card_id_list_id_map[card['id']] = course['id']

In [6]:
def getList(cardId):
    return card_id_list_id_map[cardId]

def getListName(listId):
    return list_id_course_map[listId]

In [7]:
# Get a list of all checklists
endpoint = 'https://api.trello.com/1/board/{}/checklists?key={}&token={}'.format(board, applicationKey, token)
r = requests.get(endpoint)

In [8]:
rows = []

for checklist in r.json():
    row = []
    # We only care about active tasks
    if checklist['idCard'] in card_id_task_map:
        row.append(getListName(getList(checklist['idCard']))) # course
        row.append(card_id_task_map[checklist['idCard']].replace('Exam Review: ', '')) # task
        score = int(checklist['name'].replace('[', '').replace(']', ''))
        row.append(score * len(checklist['checkItems'])) # score
        score_done = sum([1 for i in checklist['checkItems'] if i['state'] == 'complete']) * score
        row.append(score_done)
        rows.append(row)

In [9]:
df = pd.DataFrame(rows, columns=['course', 'task', 'total', 'complete'])
df

,course,task,total,complete
0,ECE 318,Problem Sets,160,160
1,ECE 318,Past Finals,90,90
2,ECE 390,Problem Sets,125,125
3,ECE 390,Past Finals,30,30
4,ECE 356,Slides,160,160
5,ECE 358,Slides,100,100
6,ECE 358,Problem Sets,80,80
7,ECE 356,Past Finals,90,90
8,ECE 358,Past Finals,90,90
9,ECE 390,Slides,80,80


In [10]:
# Persist today's dataframe to disk
todaysFile = time.strftime('data/%Y-%m-%d.csv')
if os.path.exists(todaysFile): os.remove(todaysFile)
if not os.path.exists(os.path.dirname(todaysFile)): os.makedirs(os.path.dirname(todaysFile))
df.to_csv(todaysFile, index=None)

## Plot!

In [11]:
df_daily = pd.DataFrame()
for csv in glob.glob('data/*.csv'):
    df_temp = pd.read_csv(csv)
    df_temp['date'] = csv.replace('data/', '').replace('.csv', '')
    df_daily = df_temp if df_daily.empty else df_daily.append(df_temp)

overall = df_daily.groupby(['date']).sum()
overall['remaining'] = overall['total'] - overall['complete']
overall

,total,complete,remaining
date,,,
2017-04-03,1455,0,1455
2017-04-04,1455,40,1415
2017-04-05,1455,120,1335
2017-04-06,1455,250,1205
2017-04-07,1455,250,1205
2017-04-08,1455,340,1115
2017-04-09,1455,600,855
2017-04-10,1455,750,705
2017-04-11,1455,750,705


In [12]:
overall['complete'].diff()

date
2017-04-03      NaN
2017-04-04     40.0
2017-04-05     80.0
2017-04-06    130.0
2017-04-07      0.0
2017-04-08     90.0
2017-04-09    260.0
2017-04-10    150.0
2017-04-11      0.0
2017-04-12     80.0
2017-04-13     90.0
2017-04-14     80.0
2017-04-15     70.0
2017-04-16     80.0
2017-04-17     65.0
2017-04-18     45.0
2017-04-19     70.0
2017-04-24     55.0
Name: complete, dtype: float64

In [13]:
plotly.offline.iplot({
'data': [{
    'x': overall['total'].keys(),
    'y': overall['remaining'].values
}],
'layout': {
    'title': 'Overall Burndown',
    'xaxis': {
        'range': ['2017-04-03','2017-04-21']
    },
    'yaxis': {
        'range': [0, max(overall['remaining'].values) * 1.05]
    }
}})

xaxis=dict(autorange='reversed', range=[0, 10])

In [14]:
course_burndown = df_daily.groupby(['course', 'date']).sum()
course_burndown['remaining'] = course_burndown['total'] - course_burndown['complete']
course_burndown = course_burndown.reset_index()
course_burndown.head()

,course,date,total,complete,remaining
0,EARTH 270,2017-04-03,320,0,320
1,EARTH 270,2017-04-04,320,0,320
2,EARTH 270,2017-04-05,320,0,320
3,EARTH 270,2017-04-06,320,60,260
4,EARTH 270,2017-04-07,320,60,260


In [19]:
course_burndown.groupby(['date']).sum().diff()

,total,complete,remaining
date,,,
2017-04-03,NaN,NaN,NaN
2017-04-04,0.0,40.0,-40.0
2017-04-05,0.0,80.0,-80.0
2017-04-06,0.0,130.0,-130.0
2017-04-07,0.0,0.0,0.0
2017-04-08,0.0,90.0,-90.0
2017-04-09,0.0,260.0,-260.0
2017-04-10,0.0,150.0,-150.0
2017-04-11,0.0,0.0,0.0


In [15]:
data = []
for course in course_burndown['course'].unique():
    data.append({
        'x': course_burndown[course_burndown.course == course].date,
        'y': course_burndown[course_burndown.course == course].remaining,
        'name': course
    })

# Create a list of vertical lines at exam dates, use plotly default colours
colours = ['#1F77B4','#FF7F0E','#2CA02C','#D62728','#9575D2','#8C564B','#E377C0','#7F7F7F','#BCBD22','#17BECF']
shapes = []
for course in course_burndown['course'].unique():
    shapes.append({
        'type': 'line',
        'x0': exam_dates[course],
        'y0': 0,
        'x1': exam_dates[course],
        'y1': 50,
        'line': {
            'color': colours.pop(0),
            'width': 1.5,
        }
    })

plotly.offline.iplot({
'data': data,
'layout': {
    'title': 'By Subject Burndown',
    'xaxis': {
        'range': ['2017-04-03','2017-04-21']
    },
    'yaxis': {
        'range': [0, max(course_burndown['remaining'].values) * 1.05]
    },
    'shapes': shapes
}})

xaxis=dict(autorange='reversed', range=[0, 10])